# 데이터 로드

In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [2]:
df = pd.read_csv("new_total.csv")
print(df.shape)
df.head()

(7043, 14)


,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Contract Date,Contract Period,Churn Label,Churn Score
0,8779-QRDMV,0,0,39.65,59.65,1,1,1,0,1,2023-08-14,47,1,91
1,7495-OOKFY,1,0,80.65,1024.10,8,1,0,1,0,2023-02-08,234,1,69
2,1658-BYGOY,1,0,95.45,1910.88,18,1,0,1,1,2022-04-27,521,1,81
3,4598-XLKNJ,1,0,98.50,2995.07,25,1,1,1,1,2021-09-05,755,1,88
4,4846-WHAFZ,1,0,76.50,3102.36,37,1,0,1,0,2020-09-28,1097,1,67


In [3]:
# Customer ID & 선정한 컬럼 & 'Churn Label', 'Churn Score'
selected_cols = ['Customer ID','Dependents', 'Under 30', 'Monthly Charge', 'Total Revenue', 'Tenure in Months', 'Internet Service', 
                 'Device Protection Plan', 'Unlimited Data', 'Streaming Movies', 'Churn Label', 'Churn Score']

new_df = df[selected_cols]

new_df

,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Churn Label,Churn Score
0,8779-QRDMV,0,0,39.65,59.65,1,1,1,0,1,1,91
1,7495-OOKFY,1,0,80.65,1024.10,8,1,0,1,0,1,69
2,1658-BYGOY,1,0,95.45,1910.88,18,1,0,1,1,1,81
3,4598-XLKNJ,1,0,98.50,2995.07,25,1,1,1,1,1,88
4,4846-WHAFZ,1,0,76.50,3102.36,37,1,0,1,0,1,67
...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,0,0,21.15,3039.53,72,0,0,0,0,0,45
7039,6840-RESVB,1,0,84.80,2807.47,24,1,1,1,1,0,59
7040,2234-XADUH,1,0,103.20,9453.04,72,1,1,1,1,0,71
7041,4801-JZAZL,1,0,29.60,319.21,11,1,0,1,0,0,59


# 정규화

In [4]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

scaled_df = new_df.copy()

# MinMaxScaler를 사용한 정규화
scaler = MinMaxScaler()
scaled_df[['Tenure in Months','Monthly Charge', 'Total Revenue']] = scaler.fit_transform(scaled_df[['Tenure in Months','Monthly Charge', 'Total Revenue']])

In [5]:
scaled_df

,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Churn Label,Churn Score
0,8779-QRDMV,0,0,0.212935,0.003202,0.000000,1,1,0,1,1,91
1,7495-OOKFY,1,0,0.620896,0.083855,0.098592,1,0,1,0,1,69
2,1658-BYGOY,1,0,0.768159,0.158013,0.239437,1,0,1,1,1,81
3,4598-XLKNJ,1,0,0.798507,0.248680,0.338028,1,1,1,1,1,88
4,4846-WHAFZ,1,0,0.579602,0.257652,0.507042,1,0,1,0,1,67
...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,0,0,0.028856,0.252398,1.000000,0,0,0,0,0,45
7039,6840-RESVB,1,0,0.662189,0.232992,0.323944,1,1,1,1,0,59
7040,2234-XADUH,1,0,0.845274,0.788735,1.000000,1,1,1,1,0,71
7041,4801-JZAZL,1,0,0.112935,0.024908,0.140845,1,0,1,0,0,59


# 학습과 예측에 사용할 데이터셋 만들기

In [6]:
X = scaled_df.iloc[:,1:-2]
print(X.shape)
X

(7043, 9)


,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies
0,0,0,0.212935,0.003202,0.000000,1,1,0,1
1,1,0,0.620896,0.083855,0.098592,1,0,1,0
2,1,0,0.768159,0.158013,0.239437,1,0,1,1
3,1,0,0.798507,0.248680,0.338028,1,1,1,1
4,1,0,0.579602,0.257652,0.507042,1,0,1,0
...,...,...,...,...,...,...,...,...,...
7038,0,0,0.028856,0.252398,1.000000,0,0,0,0
7039,1,0,0.662189,0.232992,0.323944,1,1,1,1
7040,1,0,0.845274,0.788735,1.000000,1,1,1,1
7041,1,0,0.112935,0.024908,0.140845,1,0,1,0


In [7]:
y = scaled_df["Churn Label"]
print(y.shape)
y

(7043,)


0       1
1       1
2       1
3       1
4       1
       ..
7038    0
7039    0
7040    0
7041    0
7042    0
Name: Churn Label, Length: 7043, dtype: int64

In [8]:
# 사이킷런에서 제공하는 model_selection의 train_test_split으로 만들기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, test_size=0.2, random_state=42)

In [9]:
# train 세트의 데이터 수를 확인
X_train.shape, y_train.shape

((5634, 9), (5634,))

In [10]:
# test 세트의 데이터 수를 확인
X_test.shape, y_test.shape

((1409, 9), (1409,))

# 여러 모델 학습

## DecisionTree, RandomForest, GradientBoosting

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42),
             RandomForestClassifier(random_state=42),
             GradientBoostingClassifier(random_state=42)]
estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [12]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2,20,10) # 의사결정트리 모델의 최대 깊이 하이퍼파라미터에 대한 후보값
max_features = np.random.uniform(0.4, 1.0, 20) # 의사결정트리 모델의 최대 피처(특성) 사용 비율 하이퍼파라미터에 대한 후보값

param_distributions = {"max_depth":max_depth, "max_features":max_features}

results = []
for estimator in estimators:
    result = []
    
    # 의사결정트리 분류기가 아닌 경우, 
    # 다른 모델의 하이퍼파라미터인 n_estimators에 대한 후보값을 설정
    if estimator.__class__.__name__ != 'DecisionTreeClassifier':
        param_distributions["n_estimators"] = np.random.randint(100, 200, 30)
        
    # 랜덤 서치 교차 검증 (성능 평가 지표:정확도)
    clf = RandomizedSearchCV(estimator, param_distributions, n_iter=100, 
                       scoring="accuracy", n_jobs=-1, cv=5, verbose=2)
    
    clf.fit(X_train, y_train)
    
    # 결과들을 dataframe에 저장
    result.append(estimator.__class__.__name__)
    result.append(clf.best_params_)
    result.append(clf.best_score_) # train dataset에서의 점수
    result.append(clf.score(X_test, y_test)) # test dataset에서의 점수
    result.append(clf.cv_results_)

    results.append(result)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [13]:
result_df = pd.DataFrame(results, columns = ["estimator", "best_params", "train_score",
                                "test_score", "cv_result"])
result_df

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.7421254487342366, 'max_dept...",0.798897,0.777147,"{'mean_fit_time': [0.016130352020263673, 0.025..."
1,RandomForestClassifier,"{'n_estimators': 158, 'max_features': 0.437281...",0.801028,0.781405,"{'mean_fit_time': [3.086930990219116, 0.711132..."
2,GradientBoostingClassifier,"{'n_estimators': 151, 'max_features': 0.497038...",0.798366,0.783534,"{'mean_fit_time': [2.383187770843506, 3.580493..."


In [14]:
# GradientBoostingClassifier 결과만 자세히 보기
pd.DataFrame(result_df.loc[2, "cv_result"]).sort_values(by="rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
50,0.799692,0.420923,0.011562,1.069014e-02,151,0.497039,2,"{'n_estimators': 151, 'max_features': 0.497038...",0.793256,0.805679,0.797693,0.801242,0.793961,0.798366,0.004641,1
63,0.967796,0.736248,0.006977,2.336015e-07,167,0.468283,2,"{'n_estimators': 167, 'max_features': 0.468282...",0.795031,0.803904,0.795031,0.802130,0.793073,0.797834,0.004329,2
80,0.515803,0.016419,0.005980,3.814697e-07,136,0.497039,2,"{'n_estimators': 136, 'max_features': 0.497038...",0.793256,0.804791,0.798580,0.801242,0.790409,0.797656,0.005225,3
10,0.719996,0.011891,0.006778,3.989022e-04,167,0.561398,2,"{'n_estimators': 167, 'max_features': 0.561397...",0.792369,0.808341,0.796806,0.796806,0.792185,0.797301,0.005880,4
15,0.941706,0.416158,0.012062,1.017084e-02,169,0.551102,2,"{'n_estimators': 169, 'max_features': 0.551101...",0.794144,0.803904,0.794144,0.802130,0.792185,0.797301,0.004755,4


## 로지스틱회귀

In [15]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [16]:
y_predict = model.predict(X_test)
y_predict

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [17]:
diff_count = (y_predict != y_test).sum() 
diff_count

316

In [18]:
accuracy_score(y_test, y_predict)

NameError: name 'accuracy_score' is not defined

## XGBoost

In [19]:
!pip install xgboost
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

XG_clf = XGBClassifier(random_state=101)

# XGBoost 학습/예측/평가
XG_clf.fit(X_train, y_train)
XG_pred = XG_clf.predict(X_test)
print('XGBoost 정확도 : {0:.4f}'.format(accuracy_score(y_test, XG_pred)))

NameError: name 'accuracy_score' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100,200,300,400],
    'learning_rate' : [0.1, 0.2, 0.3, 0.4],
    'max_depth' : [1,2,3,4,5]
    
}

xg_grid_cv = GridSearchCV(XG_clf , param_grid=params , cv=2, n_jobs=-1 )
xg_grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', xg_grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(xg_grid_cv.best_score_))

In [ ]:
XG_pred

In [ ]:
# 최적 모델에서 피처 중요도 얻기
best_model = xg_grid_cv.best_estimator_  # 최적 모델 가져오기
ftr_importances_values = best_model.feature_importances_  # 피처 중요도 얻기

# 나머지 코드는 피처 중요도 시각화를 위한 부분
ftr_importances = pd.Series(ftr_importances_values, index=X_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8, 6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig('rf_feature_importances_top20.tif', format='tif', dpi=300, bbox_inches='tight')

In [ ]:
gb_clf = GradientBoostingClassifier(random_state=101)

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100], # 결정 트리 개수 지정
    'max_depth' : [6, 8, 10, 12], # 트리의 최대 깊이 지정
    'min_samples_leaf' : [8, 12, 18,20,24 ], # 분할된 경우 왼족, 오른쪽의 브랜치 노드에서 가져가야 할 최소한의 샘플 데이터 수
    'min_samples_split' : [8, 16, 20,22,26] # 노드를 분할하기 위한 최소한의 샘플 데이터 수 => 과적합 제어
}


gb_grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2, n_jobs=-1 )
gb_grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', gb_grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(gb_grid_cv.best_score_))

# 가상 데이터

In [ ]:
test_data = [[0,0,100,7000,60,1,1,1,1]]
print('이탈 확률: ', round(xg_grid_cv.predict_proba(test_data)[0][1]*100, 2), '%')